In [73]:
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import json
%matplotlib inline
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

pd.options.display.max_columns = 50
pd.options.display.max_colwidth = 300

In [6]:
import os
%pwd

u'/Users/Driz/data_sci_wk3'

In [12]:
data = pd.read_csv('../DSI-CHI-1/lessons/week-04/3.2-logistic-regression-lab/assets/datasets/train.tsv',sep='\t', na_values={'is_news' : '?'}).fillna(0)

In [20]:
data.head()

,url,urlid,boilerplate,alchemy_category,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body
0,http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html,4042,"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at...",business,0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1.0,1,24,0,5424,170,8,0.152941,0.079130,0,"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries",A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose Califor...
1,http://www.popsci.com/technology/article/2012-07/electronic-futuristic-starting-gun-eliminates-advantages-races,8471,"{""title"":""The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races"",""body"":""And that can be carried on a pla...",recreation,0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1.0,1,40,0,4973,187,9,0.181818,0.125448,1,"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races",And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a tr...
2,http://www.menshealth.com/health/flu-fighting-fruits?cm_mmc=Facebook-_-MensHealth-_-Content-Health-_-FightFluWithFruit,1164,"{""title"":""Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health"",""body"":""Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent hea...",health,0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1.0,1,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health,Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent heart disease and cancer Next Papayas With 250 percent of the RDA of vitamin C a papaya can help kick ...
3,http://www.dumblittleman.com/2007/12/10-foolproof-tips-for-better-sleep.html,6684,"{""title"":""10 Foolproof Tips for Better Sleep "",""body"":""There was a period in my life when I had a lot of problems with sleep It took me very long to fall asleep I was easily awaken and I simply wasn t getting enough of rest at night I didn t want to take medication and this led me to learn sever...",health,0.801248,1.543103,0.400000,0.100000,0.016667,0.000000,0.480725,0.0,0,0.095861,0,0.265656,0.035343,1.0,0,24,0,2737,1

In [14]:
data.columns

Index([u'url', u'urlid', u'boilerplate', u'alchemy_category',
       u'alchemy_category_score', u'avglinksize', u'commonlinkratio_1',
       u'commonlinkratio_2', u'commonlinkratio_3', u'commonlinkratio_4',
       u'compression_ratio', u'embed_ratio', u'framebased', u'frameTagRatio',
       u'hasDomainLink', u'html_ratio', u'image_ratio', u'is_news',
       u'lengthyLinkDomain', u'linkwordscore', u'news_front_page',
       u'non_markup_alphanum_characters', u'numberOfLinks', u'numwords_in_url',
       u'parametrizedLinkRatio', u'spelling_errors_ratio', u'label'],
      dtype='object')

In [15]:
# Extract the title and body from the boilerplate JSON text
data['title'] = data.boilerplate.map(lambda x: json.loads(x).get('title', ''))
data['body'] = data.boilerplate.map(lambda x: json.loads(x).get('body', ''))

In [18]:
data[['title', 'body']].head(3)

,title,body
0,"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries",A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose Califor...
1,"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races",And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a tr...
2,Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health,Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent heart disease and cancer Next Papayas With 250 percent of the RDA of vitamin C a papaya can help kick ...


In [25]:
data['is_news'].value_counts()

1.0    4552
0.0    2843
Name: is_news, dtype: int64

In [31]:
data['alchemy_category']

0                 business
1               recreation
2                   health
3                   health
4                   sports
5                        ?
6       arts_entertainment
7                        ?
8                        ?
9                        ?
10                business
11                  sports
12                  health
13                       ?
14              recreation
15              recreation
16              recreation
17      arts_entertainment
18              recreation
19                  health
20                  sports
21              recreation
22                  health
23                       ?
24                       ?
25                  sports
26                  sports
27                business
28                       ?
29                       ?
               ...        
7365                     ?
7366                     ?
7367                     ?
7368                     ?
7369                     ?
7370            recreation
7

In [33]:
data[['is_news', 'alchemy_category']].describe()

,is_news
count,7395.000000
mean,0.615551
std,0.486498
min,0.000000
25%,0.000000
50%,1.000000
75%,1.000000
max,1.000000


In [36]:
data['is_news'].isnull().any()

False

In [41]:
data['label'].isnull().any()

False

In [42]:
logit = LogisticRegression()

In [47]:
#I want to predict a sites 'Evergreen-ness', does it being a news site predict evergreenness?
y = data['label']
x = data['is_news'].reshape(-1,1)

In [48]:
logit.fit(x,y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [50]:
logit.score(x,y)

0.5133198106828939

In [54]:
predictions = logit.predict(x)
predictions[0:]

array([1, 1, 1, ..., 1, 1, 1])

In [56]:
predict_proba = logit.predict_proba(x)
predict_proba[0:]

array([[ 0.48309259,  0.51690741],
       [ 0.48309259,  0.51690741],
       [ 0.48309259,  0.51690741],
       ..., 
       [ 0.49243515,  0.50756485],
       [ 0.48309259,  0.51690741],
       [ 0.49243515,  0.50756485]])

In [77]:
y = data['label']
x = data['is_news']

logit = sm.Logit(y, x) #set up a STATSMODELS Logistic Regression
result = logit.fit() # FIT your Logistic Regression to your data

print result.summary() # See the results of your Logistic Regression
# predictions = result.predict(x) # Get predictions for x's

Optimization terminated successfully.
         Current function value: 0.692795
         Iterations 3
                           Logit Regression Results                           
Dep. Variable:                  label   No. Observations:                 7395
Model:                          Logit   Df Residuals:                     7394
Method:                           MLE   Df Model:                            0
Date:                Wed, 19 Oct 2016   Pseudo R-squ.:              -3.671e-06
Time:                        12:42:08   Log-Likelihood:                -5123.2
converged:                       True   LL-Null:                       -5123.2
                                        LLR p-value:                       nan
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
is_news        0.0677      0.030      2.282      0.022         0.010     0.126


In [ ]:
# logit = LogisticRegression()

# y = df['is_red']
# x = df['total_sulfur_dioxide'].reshape(-1, 1)

# logit.fit(x, y)
# logit.score(x, y)
# predictions = logit.predict(x)
# predictions[0:5]
# predict_proba = logit.predict_proba(x)
# predict_proba[0:5]

In [ ]:
#STATS MODEL VERSION

# y = WHATEVER_Y_YOU_WANT
# x = WHATEVER_X_YOU_WANT

# logit = sm.Logit(y, x) #set up a STATSMODELS Logistic Regression
# result = logit.fit() # FIT your Logistic Regression to your data

# print result.summary() # See the results of your Logistic Regression
# predictions = result.predict(x) # Get predictions for x's

In [65]:
#need to change column values to dummies before running logistic regression

In [61]:
#now I want to replace this column in my original table with my dummie value table
dum_data = pd.get_dummies(data, columns = ['alchemy_category'])

In [78]:
.join?

Object `join` not found.


In [79]:
.join?

Object `join` not found.


In [62]:
dum_data.head()

,url,urlid,boilerplate,alchemy_category_score,avglinksize,commonlinkratio_1,commonlinkratio_2,commonlinkratio_3,commonlinkratio_4,compression_ratio,embed_ratio,framebased,frameTagRatio,hasDomainLink,html_ratio,image_ratio,is_news,lengthyLinkDomain,linkwordscore,news_front_page,non_markup_alphanum_characters,numberOfLinks,numwords_in_url,parametrizedLinkRatio,spelling_errors_ratio,label,title,body,alchemy_category_?,alchemy_category_arts_entertainment,alchemy_category_business,alchemy_category_computer_internet,alchemy_category_culture_politics,alchemy_category_gaming,alchemy_category_health,alchemy_category_law_crime,alchemy_category_recreation,alchemy_category_religion,alchemy_category_science_technology,alchemy_category_sports,alchemy_category_unknown,alchemy_category_weather
0,http://www.bloomberg.com/news/2010-12-23/ibm-predicts-holographic-calls-air-breathing-batteries-by-2015.html,4042,"{""title"":""IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries"",""body"":""A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at...",0.789131,2.055556,0.676471,0.205882,0.047059,0.023529,0.443783,0.0,0,0.090774,0,0.245831,0.003883,1.0,1,24,0,5424,170,8,0.152941,0.079130,0,"IBM Sees Holographic Calls Air Breathing Batteries ibm sees holographic calls, air-breathing batteries",A sign stands outside the International Business Machines Corp IBM Almaden Research Center campus in San Jose California Photographer Tony Avelar Bloomberg Buildings stand at the International Business Machines Corp IBM Almaden Research Center campus in the Santa Teresa Hills of San Jose Califor...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,http://www.popsci.com/technology/article/2012-07/electronic-futuristic-starting-gun-eliminates-advantages-races,8471,"{""title"":""The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races"",""body"":""And that can be carried on a pla...",0.574147,3.677966,0.508021,0.288770,0.213904,0.144385,0.468649,0.0,0,0.098707,0,0.203490,0.088652,1.0,1,40,0,4973,187,9,0.181818,0.125448,1,"The Fully Electronic Futuristic Starting Gun That Eliminates Advantages in Races the fully electronic, futuristic starting gun that eliminates advantages in races the fully electronic, futuristic starting gun that eliminates advantages in races",And that can be carried on a plane without the hassle too The Omega E Gun Starting Pistol Omega It s easy to take for granted just how insanely close some Olympic races are and how much the minutiae of it all can matter The perfect example is the traditional starting gun Seems easy You pull a tr...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,http://www.menshealth.com/health/flu-fighting-fruits?cm_mmc=Facebook-_-MensHealth-_-Content-Health-_-FightFluWithFruit,1164,"{""title"":""Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health"",""body"":""Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent hea...",0.996526,2.382883,0.562016,0.321705,0.120155,0.042636,0.525448,0.0,0,0.072448,0,0.226402,0.120536,1.0,1,55,0,2240,258,11,0.166667,0.057613,1,Fruits that Fight the Flu fruits that fight the flu | cold & flu | men's health,Apples The most popular source of antioxidants in our diet one apple has an antioxidant effect equivalent to 1 500 mg of vitamin C Apples are loaded with protective flavonoids which may prevent heart disease and cancer Next Papayas With 250 percent of the RDA of vitamin C a papaya can help kick ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,

In [69]:
results = dum_data.corr()

In [72]:
results['label']

urlid                                  0.013450
avglinksize                            0.006172
commonlinkratio_1                      0.083364
commonlinkratio_2                      0.083488
commonlinkratio_3                      0.105964
commonlinkratio_4                      0.080464
compression_ratio                     -0.059737
embed_ratio                            0.039536
framebased                                  NaN
frameTagRatio                         -0.187762
hasDomainLink                         -0.004863
html_ratio                            -0.051149
image_ratio                           -0.017266
is_news                                0.009103
lengthyLinkDomain                      0.032824
linkwordscore                         -0.173800
non_markup_alphanum_characters         0.097580
numberOfLinks                          0.080187
numwords_in_url                       -0.024823
parametrizedLinkRatio                  0.010668
spelling_errors_ratio                 -0

In [71]:
#want to make pivot table showing labels correlation to various columns to find the best ones to test
pd.pivot_table(results(index=['label']))

TypeError: 'DataFrame' object is not callable

In [ ]:
# To find accuracy!

logit = LogisticRegression()
model = logit.fit(x, y)
print model.score(x, y) # This is your accuracy!

from sklearn.metrics import confusion_matrix, classification_report

# Confusion Matrix!

y_pred = model.predict(x)
confusion = np.array(confusion_matrix(y, y_pred)) 

# If you want a pretty confusion matrix:

conmat = np.array(confusion_matrix(y, y_pred))

confusion = pd.DataFrame(conmat, index=['Y=0', 'Y=1'],
                         columns=['Y-hat = 0', 'Y-hat = 1'])

# Precision and Recall!

classifcation_report(y, y_pred)

In [84]:
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

In [96]:
y = data.label
y.shape

(7395,)

In [106]:
dum_data['label'].isnull().any()

False

In [119]:
dum_data.columns

Index([u'url', u'urlid', u'boilerplate', u'alchemy_category_score',
       u'avglinksize', u'commonlinkratio_1', u'commonlinkratio_2',
       u'commonlinkratio_3', u'commonlinkratio_4', u'compression_ratio',
       u'embed_ratio', u'framebased', u'frameTagRatio', u'hasDomainLink',
       u'html_ratio', u'image_ratio', u'is_news', u'lengthyLinkDomain',
       u'linkwordscore', u'news_front_page', u'non_markup_alphanum_characters',
       u'numberOfLinks', u'numwords_in_url', u'parametrizedLinkRatio',
       u'spelling_errors_ratio', u'label', u'title', u'body',
       u'alchemy_category_?', u'alchemy_category_arts_entertainment',
       u'alchemy_category_business', u'alchemy_category_computer_internet',
       u'alchemy_category_culture_politics', u'alchemy_category_gaming',
       u'alchemy_category_health', u'alchemy_category_law_crime',
       u'alchemy_category_recreation', u'alchemy_category_religion',
       u'alchemy_category_science_technology', u'alchemy_category_sports',
    

In [109]:
logit = LogisticRegression()

In [125]:
y = dum_data['label']
x = dum_data[['alchemy_category_arts_entertainment','alchemy_category_business', 
             'alchemy_category_computer_internet','alchemy_category_culture_politics', 
             'alchemy_category_gaming','alchemy_category_health', 'alchemy_category_law_crime',
             'alchemy_category_recreation', 'alchemy_category_religion',
             'alchemy_category_science_technology', 'alchemy_category_sports',
             'alchemy_category_weather']]

In [131]:
model = logit.fit(x,y)
model.score(x,y)

0.6093306288032454

In [134]:
# Confusion Matrix!
y_pred = model.predict(x)
confusion = np.array(confusion_matrix(y, y_pred))
print confusion

[[1571 2028]
 [ 861 2935]]


In [135]:
# If you want a pretty confusion matrix:

conmat = np.array(confusion_matrix(y, y_pred))

confusion = pd.DataFrame(conmat, index=['Y=0', 'Y=1'],
                         columns=['Y-hat = 0', 'Y-hat = 1'])


In [136]:
print confusion

     Y-hat = 0  Y-hat = 1
Y=0       1571       2028
Y=1        861       2935


In [138]:
# Precision and Recall!

classification_report(y, y_pred)

'             precision    recall  f1-score   support\n\n          0       0.65      0.44      0.52      3599\n          1       0.59      0.77      0.67      3796\n\navg / total       0.62      0.61      0.60      7395\n'